In [1]:
from bqplot import pyplot as plt
from bqplot import Lines, LinearScale, ColorScale, Tooltip, Scatter
import ipywidgets as widgets
import numpy as np
import pandas as pd
from ipywidgets import Output
from IPython.display import display, clear_output, Markdown

def pprint(*args):
    return display(Markdown(*args))
pprint("""
# Basic data analysis of Madrid COVID-19 incidence
""")


# Basic data analysis of Madrid COVID-19 incidence


In [2]:
#!wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/b7b9edb4-0c70-47d3-9c64-8c4913830a24/download/covid19_tia_zonas_basicas_salud.csv 2> /dev/null
!wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv 2> /dev/null
#!tail -n +2 covid19_tia_zonas_basicas_salud.csv > covid19_tia_zonas_basicas_salud2.csv
!touch covid19_tia_zonas_basicas_salud2.csv
!cat covid19_tia_zonas_basicas_salud_s.csv covid19_tia_zonas_basicas_salud2.csv > covid19_tia_zonas_basicas_salud.csv

In [7]:
b = widgets.Button(description="Reload data")
def downloadData(_):
    !rm *.csv
    #!wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/b7b9edb4-0c70-47d3-9c64-8c4913830a24/download/covid19_tia_zonas_basicas_salud.csv 2> /dev/null
    !wget -nc https://datos.comunidad.madrid/catalogo/dataset/b3d55e40-8263-4c0b-827d-2bb23b5e7bab/resource/43708c23-2b77-48fd-9986-fa97691a2d59/download/covid19_tia_zonas_basicas_salud_s.csv 2> /dev/null
    !touch covid19_tia_zonas_basicas_salud2.csv
    #!tail -n +2 covid19_tia_zonas_basicas_salud.csv > covid19_tia_zonas_basicas_salud2.csv
    !cat covid19_tia_zonas_basicas_salud_s.csv covid19_tia_zonas_basicas_salud2.csv > covid19_tia_zonas_basicas_salud.csv
b.on_click(downloadData)
pprint("Click this button to reload the data and update the site")
b

Click this button to reload the data and update the site

Button(description='Reload data', style=ButtonStyle())

In [4]:
df = pd.read_csv("covid19_tia_zonas_basicas_salud.csv", sep=";", encoding='latin-1')
df["date"] = pd.to_datetime(df.fecha_informe)
df.tasa_incidencia_acumulada_ultimos_14dias = pd.to_numeric(df.tasa_incidencia_acumulada_ultimos_14dias.str.replace(",", "."))

In [5]:
d = pd.DataFrame([df.groupby("zona_basica_salud").nth(0).tasa_incidencia_acumulada_ultimos_14dias, df.groupby("zona_basica_salud").nth(1).tasa_incidencia_acumulada_ultimos_14dias, df.groupby("zona_basica_salud").nth(0).fecha_informe, df.groupby("zona_basica_salud").nth(1).fecha_informe])
d = d.T
d.columns = ["Incidencia esta semana", "Incidencia semana anterior", "Última fecha de informe", "Fecha de informe anterior"]
d["Variación"] = (d["Incidencia esta semana"] - d["Incidencia semana anterior"])*100 / d["Incidencia semana anterior"]
d["Variación"] = d["Variación"].apply(lambda e: round(e, 2))
d["color_var"] = d["Variación"].apply(lambda e: "red" if e>100 else "orange" if e > 50 else "yellow" if e > 0 else "green" if e > -50 else "blue")
d["variacion"] = d["Variación"]

d = d.reset_index()
import ipyvuetify as v
from traitlets import (Unicode, List)
import pandas as pd

class MyDataTable(v.VuetifyTemplate):
    
    headers = List([
        {'text': 'Zona Básica de Salud', 'sortable':  True, 'value': 'zona_basica_salud'},
        {'text': 'Incidencia esta semana', 'value': "Incidencia esta semana", 'align': 'right'},
        {'text': 'Incidencia semana anterior', 'value': "Incidencia semana anterior", 'align': 'right'},
        {'text': 'Última fecha de informe', 'value': "Última fecha de informe", 'align': 'right'},
        {'text': 'Fecha de informe anterior', 'value': "Fecha de informe anterior", 'align': 'right'},
        {'text': 'Variación (%)', 'value': "variacion", 'align': 'right'}
    ]).tag(sync=True)
    d1 = []
    for a in (d.to_dict(orient='records')):
        if a["Incidencia esta semana"] > 1000:
            a["variant"] = "danger"
        d1.append(a)
    items = List(d1).tag(sync=True)
    template = Unicode('''
        <v-data-table
            :headers="headers"
            :items="items"
            class="elevation-1" >
        <template v-slot:item.variacion="{ item }">
      <v-chip
        :color="item.color_var">
        {{ item.variacion }}
      </v-chip>
    </template>
        </v-data-table>
    ''').tag(sync=True)
    
    def vue_getColor(self, e):
        print(e)
        return "red" if e>100 else "orange" if e > 50 else "yellow" if e > 0 else "blue"
d = MyDataTable()
d

MyDataTable(events=['getColor'], headers=[{'text': 'Zona Básica de Salud', 'sortable': True, 'value': 'zona_ba…

In [6]:
f = plt.figure(title="Incidencia")
zonas = np.unique(df.zona_basica_salud)

#zones = np.unique(df.zona_basica_salud).tolist()
zones = ["Peñagrande", "Daroca"]
fdf = df[df.zona_basica_salud.isin(zones)]
data = [(zbs, partdf) for zbs, partdf in fdf.groupby("zona_basica_salud")]
ZBSs=[e[0] for e in data]
dfs = [e[1] for e in data]
xs = [d.date for d in dfs]
ys = [d.tasa_incidencia_acumulada_ultimos_14dias for d in dfs]
names = [d.zona_basica_salud for d in dfs]
x, y, names = np.stack(xs).reshape(-1), np.stack(ys).reshape(-1), np.stack(names).reshape(-1).tolist()



p1 = plt.plot(xs, ys, labels=ZBSs)
plt.legend()
colors = [[p1.colors[i % len(p1.colors)] for _ in range(len(d))] for i, d in enumerate(xs)] 
colors = np.stack(colors).reshape(-1).tolist()

p2 = plt.scatter(x=x,y=y, names=names, labels=names, display_names=False, colors=colors)
p2.tooltip = Tooltip(fields=['name','x', 'y'], formats = ['',"%d-%b-%y",''], labels = ['ZBS','Fecha', 'Incidencia'], show_labels = True )
p2.enable_hover = True


def on_change(args):
    global zones
    if args["new"] == True:
        zones = zones + [args["owner"].description]
    else:
        zones.remove(args["owner"].description)
    print(zones)

    fdf = df[df.zona_basica_salud.isin(zones)]
    data = [(zbs, partdf) for zbs, partdf in fdf.groupby("zona_basica_salud")]
    ZBSs=[e[0] for e in data]
    dfs = [e[1] for e in data]
    xs = [d.date for d in dfs]
    ys = [d.tasa_incidencia_acumulada_ultimos_14dias for d in dfs]
    names = [d.zona_basica_salud for d in dfs]
    x, y, names = np.stack(xs).reshape(-1), np.stack(ys).reshape(-1), np.stack(names).reshape(-1).tolist()
    
    p1.x = xs
    p1.y = ys
    p1.labels = ZBSs
    colors = [[p1.colors[i % len(p1.colors)] for _ in range(len(d))] for i, d in enumerate(xs)] 
    colors = np.stack(colors).reshape(-1).tolist()
    
    p2.x = x
    p2.y = y
    p2.names = names
    p2.labels = names
    p2.colors = colors
    p2.tooltip = Tooltip(fields=['name','x', 'y'], formats = ['',"%d-%b-%y",''], labels = ['ZBS','Fecha', 'Incidencia'], show_labels = True )
    p2.enable_hover = True
ws = [widgets.Checkbox(value=zone in zones, description=zone) for zone in zonas]
[w.observe(on_change, "value") for w in ws]
widgets.VBox([f, *ws])
